In [2]:
# This script processes a gejson file and splits it into the directory format needed by the website to work
# It should be ran in QGIS.

from qgis.core import *
from qgis.utils import iface


# Code to import csv file, change variables as needed
InFlnm='outfile.csv'
InDrPth='C:/programming/MHGIS/workfile/'
InFlPth="file:///"+InDrPth+InFlnm
uri = InFlPth+"?delimiter=%s&xField=%s&yField=%s" % (",","longitude","latitude")

url = "file:///C:/programming/MHGIS/workfile/outfile.csv"
bh = QgsVectorLayer(uri, InFlnm, "delimitedtext")

bh.isValid()
bh.setCrs(QgsCoordinateReferenceSystem(4326, QgsCoordinateReferenceSystem.EpsgCrsId))
QgsProject.instance().addMapLayer(bh)


processing.run('qgis:splitvectorlayer', {'INPUT':"outfile.csv", 'FIELD':"region", 'FILE_TYPE':7, 'OUTPUT':"C:/programming/MHGIS/workfile/categorised_master"})

ModuleNotFoundError: No module named 'qgis'

In [3]:
# Part 2, run here
# 

import json, os

for x in range(14):
    
    indir = './categorised_master/region_'+str(x)+'.geojson'
    
    cishanfuli = []
        
    with open(indir) as infile:
        data = json.load(infile)

        # Rename files
        region = data["features"][0]["properties"]["Region"]
        outdir = './categorised_master/'+region+'.geojson'

        # Dump to renamed file
        with open(outdir, 'w') as outfile:
            json.dump(data, outfile)
            
        # Create directory if it does not exist 
        outsplitdir = './categorised_master/'+region+'/'
        if not os.path.exists(outsplitdir):
            os.makedirs(outsplitdir)
            
        
              

In [7]:
# Part 3, run in Qgis

from qgis.core import *

layername = ""
states=["Johor", "Kedah", "Kelantan", "Kuala Lumpur", "Melaka", "Negeri Sembilan", "Pahang", "Penang", "Perak", "Perlis", "Sabah", "Sarawak", "Selangor", "Terengganu"]

for state in states:
    InDrPth='C:/programming/MHGIS/workfile/categorised_master/'+state+'.geojson'
    layer = QgsVectorLayer(InDrPth,state,"ogr")
    QgsProject.instance().addMapLayer(layer)
    
    outdir="C:/programming/MHGIS/workfile/categorised_master/"+state+"/"
    processing.run('qgis:splitvectorlayer', {'INPUT':layer, 'FIELD':"Category", 'FILE_TYPE':7, 'OUTPUT':outdir})

ModuleNotFoundError: No module named 'qgis'

In [6]:
# Part 4, run here
import os, json

states=["Johor", "Kedah", "Kelantan", "Kuala Lumpur", "Melaka", "Negeri Sembilan", "Pahang", "Penang", "Perak", "Perlis", "Sabah", "Sarawak", "Selangor", "Terengganu"]

key = {"慈善福利": "cishanfuli", "地缘": "diyuan", "独立中学": "dulizhongxue", "青年妇女" : "qingniaofunu", "体育联谊": "tiyulianyi", "文教": "wenjiao", "血缘": "xueyuan", "业缘": "yeyuan", "综合": "zonghe", "宗教": "zongjiao"}
       

for state in states:
    indir="./categorised_master/"+state+"/"
    
    
    for i in range(10):
        inpath="./categorised_master/"+state+"/category_"+str(i)+".geojson"
        if os.path.exists(inpath):
            with open(inpath) as infile:
                data = json.load(infile)
                category = data["features"][0]["properties"]["Category"]
                category_pinyin = key[category]
                
            outpath="./categorised_master/"+state+"/"+category_pinyin+".json"
            os.rename(inpath, outpath)
    
                
        else:
            continue
        
    
    

In [6]:
# Geocoding temples

import json, geocoder, csv

outbuffer = []

with open('temple.json') as infile:
    indata = json.load(infile)
    for entry in indata:
        if entry["Address"] != "":
            g = geocoder.google(entry["Address"], key = "[[INSERT WHEN NEEDED, REMOVE BEFORE COMMIT]]")
            print(g)
        
        if entry["Address"] == "" or g.latlng == None:
            outbuffer.append( {
                "Name": entry["Name"], 
                "Notes": entry["Notes"], 
                "Address": entry["Address"], 
                "State": entry["State"], 
                "UID": entry["UID"],
                "Latitude": 0 , 
                "Longitude": 0
            })
        else: 
            outbuffer.append({
                "Name": entry["Name"], 
                "Notes": entry["Notes"], 
                "Address": entry["Address"], 
                "State": entry["State"], 
                "UID": entry["UID"],
                "Latitude": g.latlng[0] , 
                "Longitude": g.latlng[1]
            })
            
print(outbuffer)

<[OK] Google - Geocode [Taman Sembrong Barat, 83700 Yong Peng, Johor, Malaysia]>
<[OK] Google - Geocode [31, Jalan Ang Moh Sam, Taman Sembrong Baru, 83700 Yong Peng, Johor, Malaysia]>
<[OK] Google - Geocode [Taman Selatan, 83700 Yong Peng, Johor, Malaysia]>
<[OK] Google - Geocode [Jalan Aman, Taman Sembrong Baru, 83700 Yong Peng, Johor, Malaysia]>
<[OK] Google - Geocode [Jalan Madu 1, Taman Sembrong Baru, 83700 Yong Peng, Johor, Malaysia]>
<[OK] Google - Geocode [83700, Johor, Malaysia]>
<[OK] Google - Geocode #2 results>
<[OK] Google - Geocode [4, Jalan Ceria 19, Taman Nusa Indah, 81200 Johor Bahru, Johor, Malaysia]>
<[OK] Google - Geocode [Jalan Persiaran Pertama, Taman Tampoi Indah 2, 81300 Johor Bahru, Johor, Malaysia]>
<[OK] Google - Geocode [54, Jalan Ronggeng 19, Taman Nesa, 81300 Skudai, Johor, Malaysia]>
<[OK] Google - Geocode [2, Jalan Temenggong 9, Taman Ungku Tun Aminah, 81300 Skudai, Johor, Malaysia]>
<[OK] Google - Geocode [83700, Johor, Malaysia]>
<[OK] Google - Geocode 

<[OK] Google - Geocode [Jalan Anggerik 1, Taman Kulai Utama, 81000 Kulai, Johor, Malaysia]>
<[OK] Google - Geocode [525, Jalan Serunai 10, Taman Pancasila, 81400 Kulai, Johor, Malaysia]>
<[OK] Google - Geocode [245, Jalan Serunai 4, Taman Saleng, 81400 Senai, Johor, Malaysia]>
<[OK] Google - Geocode #2 results>
<[OK] Google - Geocode [Jalan Jambu 28, Taman Muhibbah, 81400 Kulai, Johor, Malaysia]>
<[OK] Google - Geocode [Jalan Sekolah Cina, 81000 Kulai, Johor, Malaysia]>
<[OK] Google - Geocode [6035, Jalan Persiaran Indahpura 3, Bandar Indahpura, 81000 Kulai, Johor, Malaysia]>
<[OK] Google - Geocode [Lorong 2, 81000 Kulai, Johor, Malaysia]>
<[OK] Google - Geocode [15, Jalan Indah 3, Taman Sawit Indah, 81000 Kulai, Johor, Malaysia]>
<[OK] Google - Geocode [19, Jalan Molek 3/19, Taman Molek, 81100 Johor Bahru, Johor, Malaysia]>
<[OK] Google - Geocode [15, Jalan Bakawali 48, Johor Jaya, 81100 Johor Bahru, Johor, Malaysia]>
<[OK] Google - Geocode [Jalan Dedap 30, Johor Jaya, 81100 Johor Bah

<[OK] Google - Geocode [2, Jalan Temenggong 9, Taman Ungku Tun Aminah, 81300 Skudai, Johor, Malaysia]>
<[OK] Google - Geocode [Lorong B-8, Pekan Nenas, 81500 Pekan Nanas, Johor, Malaysia]>
<[OK] Google - Geocode [Lorong B-5, Pekan Nenas, 81500 Pekan Nanas, Johor, Malaysia]>
<[OK] Google - Geocode [510, Lorong C-5, Taman Sri Kiambang, 81500 Pekan Nanas, Johor, Malaysia]>
<[OK] Google - Geocode [458, Lorong C-2, Taman Seri Damansara, 81500 Pekan Nanas, Johor, Malaysia]>
<[OK] Google - Geocode [Lorong B-2, Pekan Nenas, 81500 Pekan Nanas, Johor, Malaysia]>
<[OK] Google - Geocode [Pekan Nanas, 81500 Pekan Nanas, Johor, Malaysia]>
<[OK] Google - Geocode [Pekan Nanas, 81500 Pekan Nanas, Johor, Malaysia]>
<[OK] Google - Geocode [Jalan Tanjung Kupang, Kampung Ulu Choh, Pekan Nanas, Johor, Malaysia]>
<[OK] Google - Geocode [Jalan Ulu Choh, Kampung Ulu Choh, 81500 Pekan Nanas, Johor, Malaysia]>
<[OK] Google - Geocode [80, Jalan Kobis, Kampung Ulu Choh, 81500 Pekan Nanas, Johor, Malaysia]>
<[OK] G

In [7]:
print (outbuffer)

with open('temple_georefed.json','w', newline='', encoding="utf-8") as outfile:
    json.dump(outbuffer, outfile, ensure_ascii=False).encode("utf8")
        

[{'Name': '永平天保宮 (68尺濟公)', 'Notes': '', 'Address': 'Taman Sembrong Barat, 83700 Yong Peng, Johor', 'State': 'Johor', 'UID': 'A001', 'Latitude': 2.010124, 'Longitude': 103.066232}, {'Name': '永平天靈宮', 'Notes': '', 'Address': '31, Jalan Ang Moh Sam, Taman Sembrong Baru, 83700 Yong Peng, Johor', 'State': 'Johor', 'UID': 'A002', 'Latitude': 2.0180711, 'Longitude': 103.065743}, {'Name': '永平天後宮-又稱媽祖廟', 'Notes': '', 'Address': 'Taman Selatan, 83700 Yong Peng, Johor.', 'State': 'Johor', 'UID': 'A003', 'Latitude': 2.01939, 'Longitude': 103.062632}, {'Name': '正福德宮', 'Notes': '', 'Address': 'Jalan Aman, Taman Sembrong Baru, 83700 Yong Peng, Johor', 'State': 'Johor', 'UID': 'A004', 'Latitude': 2.014129, 'Longitude': 103.0671151}, {'Name': '佛教圓滿蕯迦中心', 'Notes': '', 'Address': 'Jalan Madu 1, Taman Sembrong Baru, 83700 Yong Peng, Johor, Johor', 'State': 'Johor', 'UID': 'A005', 'Latitude': 2.0141289, 'Longitude': 103.0659716}, {'Name': '永平德教會紫安閣', 'Notes': '', 'Address': '83700 Yong Peng, Johor', 'State'

AttributeError: 'NoneType' object has no attribute 'encode'

In [34]:
import csv, json

with open('./mys_adminboundaries.csv') as infile:
    indata = csv.DictReader(infile)
    
    outbuffer = []
    
    for row in indata:

        if not any(row["admin1Pcode"] == d["code"] for d in outbuffer):
            buffer_state = {
                "state_name": row["admin1Name_en"],
                "code": row["admin1Pcode"],
                "territories": []
            }
            
            outbuffer.append(buffer_state)
            
        buffer_territory = {
            "territory_name": row["admin2Name_en"],
            "code": row["admin2Pcode"]
        }
        
        for state in outbuffer:
            if state["code"] == row["admin1Pcode"]:
                state["territories"].append(buffer_territory)
                
outbuffer = sorted(outbuffer, key= lambda k: k['code'])

with open('MYterritories.json','w', newline='', encoding="utf-8") as outfile:
    json.dump(outbuffer, outfile, ensure_ascii=False)